# Курс "Машинное обучение в бизнесе"

## Практическое задание урока 6. Задача look-alike

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

### Решение

Подключение библиотек и скриптов¶

In [278]:
%matplotlib inline

import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np

In [279]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

# Функция для проверки качества
def evaluate_results(y_test, y_predict):
    f1 = f1_score(y_test, y_predict).round(4)
    roc = roc_auc_score(y_test, y_predict).round(4)
    rec = recall_score(y_test, y_predict, average='binary').round(4)
    prc = precision_score(y_test, y_predict, average='binary').round(4)
    
    return f1, roc, rec, prc

#### 1. Загрузка данных

Расмотрим пример на датасете из репозитория UCI

Описание данных - https://archive.ics.uci.edu/ml/datasets/Adult

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [280]:
df = pd.read_csv("adult.data", 
                 names=['age', 'workclass', 'fnlwgt', 'education',
                        'education-num', 'marital-status', 'occupation',
                        'relationship', 'race', 'sex', 'capital-gain',
                        'capital-loss', 'hours-per-week', 'native-country', 'income'])
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


В датасете имеются 14 признаков и 1 целевая переменная (бинарная) - нужно определить зарабатывает человек свыше 50К в год или нет.

In [281]:
print(df.shape)

(32561, 15)


Всего в датасете сведения о 32561 людях.

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


#### 2. Feature Engineering

In [283]:
for value in ['age', 'workclass', 'fnlwgt', 'education',
            'education-num', 'marital-status', 'occupation',
            'relationship', 'race', 'sex', 'capital-gain',
            'capital-loss', 'hours-per-week', 'native-country']:
    print(df[value].value_counts(), '\n--------')

36    898
31    888
34    886
23    877
35    876
     ... 
83      6
85      3
88      3
87      1
86      1
Name: age, Length: 73, dtype: int64 
--------
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64 
--------
164190    13
203488    13
123011    13
113364    12
121124    12
          ..
284211     1
312881     1
177711     1
179758     1
229376     1
Name: fnlwgt, Length: 21648, dtype: int64 
--------
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          5

Посмотрим на соотношение классов:

In [284]:
df['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [285]:
# Приведем поле целевой переменной 'income' к бинарному виду (1 или 0)
# значение ' >50K' означает 1, а все остальные - 0.
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})
df['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [286]:
df_copy = df.copy()

#### 3. Обучение классификатора

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [287]:
from sklearn.model_selection import train_test_split

x_data = pd.get_dummies(df.iloc[:, :-1])
y_data = df['income']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [288]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

# вычисляем качество
results_1 = evaluate_results(y_test, y_predict)

#### 4. PU learning с долей P 25%

Представим, что нам неизвестны негативы и часть позитивов

In [289]:
mod_data = pd.get_dummies(df_copy)
#get the indices of the positives samples
pos_ind = np.where(mod_data.loc[:,'income'].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1961/7841 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [290]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    30600
 1     1961
Name: class_test, dtype: int64


Имеем 1961 позитивных примеров со значением 1 в поле 'class_test', всё остальное - неразмеченное со значением -1.

Поле 'income' содержит фактические метки

In [291]:
mod_data.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ ?,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native-country_ ?,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,class_test
0,39,77516,13,2174,0,40,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1
1,50,83311,13,0,0,13,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1
2,38,215646,9,0,0,40,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1
3,53,234721,7,0,0,40,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1
4,28,338409,13,0,0,40,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
5,37,284582,14,0,0,40,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1
6,49,160187,5,0,0,16,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0

Помним, что этот датасет (x_data) содержит бывшую целевую переменную, которую мы оставили, чтобы сравнить результаты в поле
'income' - оригинального класса позитивных и негативных данных - и в поле 'class_test' - нового класса позитивных и неразмеченных данных.

In [292]:
x_data = mod_data.drop(['income', 'class_test'], axis=1).values # just the X 
y_labeled = mod_data.loc[:, 'class_test'].values # new class (just the P & U)
y_positive =  mod_data.loc[:, 'income'].values # original class

#### 5. random negative sampling

In [293]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 110) (1961, 110)


In [294]:
model = xgb.XGBClassifier()

model.fit(sample_train.drop(['income', 'class_test'], axis=1).values, 
          sample_train.loc[:, 'income'].values)
y_predict = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

# вычисляем качество
results_2 = evaluate_results(sample_test.loc[:, 'income'].values, y_predict)

#### 6. Сравнение качества

In [295]:
metrics = {'xgboost': results_1, 
            'random negative sampling': results_2
          }
info = pd.DataFrame(columns=metrics.keys())

In [296]:
for metric, value in metrics.items():
    print(metric, value)
    f1, roc, recall, precision = value
    info.loc[f'fscore', metric] = f1
    info.loc[f'roc_auc', metric] = roc
    info.loc[f'recall', metric] = recall
    info.loc[f'precision', metric] = precision

xgboost (0.709, 0.796, 0.6498, 0.7802)
random negative sampling (0.6116, 0.8217, 0.8831, 0.4678)


In [297]:
info

,xgboost,random negative sampling
fscore,0.709,0.6116
roc_auc,0.796,0.8217
recall,0.6498,0.8831
precision,0.7802,0.4678


#### 7. Эксперименты с долей P

In [301]:
def experiment(df, p_rate):
    
    # Представим, что нам неизвестны негативы и часть позитивов
    mod_data = pd.get_dummies(df)
    #get the indices of the positives samples
    pos_ind = np.where(mod_data.loc[:,'income'].values == 1)[0]
    #shuffle them
    np.random.shuffle(pos_ind)
    # leave just p_rate *100% of the positives marked
    pos_sample_len = int(np.ceil(p_rate * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    
    # Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    
    
    x_data = mod_data.drop(['income', 'class_test'], axis=1).values # just the X 
    y_labeled = mod_data.loc[:, 'class_test'].values # new class (just the P & U)
    y_positive =  mod_data.loc[:, 'income'].values # original class
    
    #  random negative sampling
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]

    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    
    model = xgb.XGBClassifier()

    model.fit(sample_train.drop(['income', 'class_test'], axis=1).values, 
          sample_train.loc[:, 'income'].values)
    y_predict = model.predict(sample_test.drop(['income', 'class_test'], axis=1).values)

    # вычисляем качество
        
    return evaluate_results(sample_test.loc[:, 'income'].values, y_predict)

In [310]:
for p_rate in [0.1, 0.2, 0.25, 0.3, 0.4, 0.5]:
    print("random negative sampling P %.0f%%" % (p_rate * 100.0), experiment(df_copy, p_rate))

random negative sampling P 10% (0.6373, 0.8117, 0.8687, 0.5033)
random negative sampling P 20% (0.6155, 0.8181, 0.8881, 0.4709)
random negative sampling P 25% (0.6145, 0.8237, 0.8858, 0.4704)
random negative sampling P 30% (0.5956, 0.824, 0.8936, 0.4467)
random negative sampling P 40% (0.5734, 0.8281, 0.8839, 0.4244)
random negative sampling P 50% (0.5389, 0.8316, 0.8869, 0.387)


С увеличением размера P имеется тенденция к снижению fscore, precision и незначительному возрастанию roc_auc.

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?